In [ ]:
# 회귀회로 -> 항등사항(나오는대로 쓴다) -> 제곱오차 사용
# 이진분류(0 or 1) -> 로지스틱 펑션(activation: sigmoid, tanh) -> BCEloss
# 다클래스 분류 -> softmax (다 클래스의 확률 합이 1) -> CrossEntropy
# 모두를 위한 딥러닝 강의 lec6-2 참고 하기 

In [110]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from  torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
#import visdom

import numpy as np

import torchvision     # data set 불러오기위해
import torchvision.transforms as transforms

In [31]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]) # 초기값 설정 부분 totensor - numpy를 torch로 변경
#compose - 여러개를 동시에 transforms 해준다.
trainset = datasets.CIFAR10(root = '/media/volume1/juna/practice', 
                                        train = True,
                                        download = True, 
                                        transform = transform)
testset = datasets.CIFAR10(root = '/media/volume1/juna/practice', 
                                       train = False, 
                                       download = True, 
                                       transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [59]:
trainloader = DataLoader(trainset, batch_size = 8, shuffle = True, num_workers =2)
testloader = DataLoader(testset, batch_size = 8, shuffle = False, num_workers =2)
trainloader.__dict__


{'_DataLoader__initialized': True,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x7fa3cc1fd400>,
 'batch_size': 8,
 'collate_fn': <function torch.utils.data.dataloader.default_collate(batch)>,
 'dataset': Dataset CIFAR10
     Number of datapoints: 50000
     Split: train
     Root Location: /media/volume1/juna/practice
     Transforms (if any): Compose(
                              ToTensor()
                              Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                          )
     Target Transforms (if any): None,
 'drop_last': False,
 'num_workers': 2,
 'pin_memory': False,
 'sampler': <torch.utils.data.sampler.RandomSampler at 0x7fa3cc1fdc18>,
 'timeout': 0,
 'worker_init_fn': None}

In [119]:
imgs = 0
for i, (img, labels) in enumerate (trainloader):
    imgs = img
    break

print(imgs.shape)
print(labels.shape)

conv1 = nn.Conv2d(3,64,5)
conv2 = nn.Conv2d(64,30,5)
fc1 = nn.Linear(30*5*5, 128)
fc2 = nn.Linear(128, 10)

x = F.relu(conv1(imgs), inplace =True)
print(x.shape)
x = F.max_pool2d(x,(2,2))
print(x.shape)
x = F.relu(conv2(x), inplace = True)
print(x.shape)
x = F.max_pool2d(x,(2,2))
print(x.shape)
x = x.view(x.shape[0],-1) # batchsize x 30 x 5 x5 를 batchsize x 전체size 로 바꿔준다
print(x.shape)
x = F.relu(fc1(x), inplace = True)
print(x.shape)
x = F.relu(fc2(x), inplace = True)
print(x.shape) # 최종 out shpae 이 8*10

torch.Size([8, 3, 32, 32])
torch.Size([8])
torch.Size([8, 64, 28, 28])
torch.Size([8, 64, 14, 14])
torch.Size([8, 30, 10, 10])
torch.Size([8, 30, 5, 5])
torch.Size([8, 750])
torch.Size([8, 128])
torch.Size([8, 10])


In [60]:
testloader.__dict__

{'_DataLoader__initialized': True,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x7fa3cc1fd550>,
 'batch_size': 8,
 'collate_fn': <function torch.utils.data.dataloader.default_collate(batch)>,
 'dataset': Dataset CIFAR10
     Number of datapoints: 10000
     Split: test
     Root Location: /media/volume1/juna/practice
     Transforms (if any): Compose(
                              ToTensor()
                              Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                          )
     Target Transforms (if any): None,
 'drop_last': False,
 'num_workers': 2,
 'pin_memory': False,
 'sampler': <torch.utils.data.sampler.SequentialSampler at 0x7fa3cc1fdc88>,
 'timeout': 0,
 'worker_init_fn': None}

In [118]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.conv1 = nn.Conv2d(3,64,5)
        self.conv2 = nn.Conv2d(64,30,5)
        self.fc1 = nn.Linear(30*5*5, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self,x):
        x = F.relu(self.conv1(x), inplace =True)
        x = F.max_pool2d(x,(2,2))
        x = F.relu(self.conv2(x), inplace = True)
        x = F.max_pool2d(x,(2,2))
        x = x.view(x.shape[0],-1) # batchsize x 30 x 5 x5 를 batchsize x 전체size 로 바꿔준다
        x = F.relu(self.fc1(x), inplace = True)
        x = F.relu(self.fc2(x), inplace = True)
        
        return x

my_net = model()

optim = torch.optim.SGD(my_net.parameters(),lr = 0.001,momentum=0.9)
loss_function = nn.CrossEntropyLoss() #crossentropy -> BATCH X LABELS VS BATCH 로 들어가야함

for epoch in range(1):
    for i,(inputs, labels) in enumerate(trainloader):
        inputs, labels = Variable(inputs), Variable(labels)
        
        optim.zero_grad()
        out = my_net(inputs)
        #print(out.shape)
        loss = loss_function(out,labels) # 8*10 / 8 -> loss 사용법?
        loss.backward()
        optim.step()
        
        if i % 64 ==0:
            print("%d => loss : %.3f"%(i, loss))
            
print("train over")    

0 => loss : 2.289
64 => loss : 2.296
128 => loss : 2.322
192 => loss : 2.296
256 => loss : 2.325
320 => loss : 2.335
384 => loss : 2.303
448 => loss : 2.318
512 => loss : 2.322
576 => loss : 2.336
640 => loss : 2.288
704 => loss : 2.218
768 => loss : 2.236
832 => loss : 2.244
896 => loss : 2.250
960 => loss : 2.199
1024 => loss : 2.487
1088 => loss : 2.082
1152 => loss : 2.168
1216 => loss : 2.403
1280 => loss : 2.403
1344 => loss : 2.456
1408 => loss : 2.540
1472 => loss : 2.132
1536 => loss : 1.953
1600 => loss : 2.210
1664 => loss : 1.744
1728 => loss : 2.110
1792 => loss : 1.679
1856 => loss : 2.172
1920 => loss : 1.813
1984 => loss : 1.626
2048 => loss : 2.732
2112 => loss : 2.342
2176 => loss : 1.973
2240 => loss : 2.648
2304 => loss : 2.168
2368 => loss : 1.862
2432 => loss : 1.960
2496 => loss : 1.683
2560 => loss : 2.002
2624 => loss : 1.776
2688 => loss : 1.936
2752 => loss : 1.334
2816 => loss : 1.798
2880 => loss : 2.217
2944 => loss : 1.716
3008 => loss : 1.553
3072 => los

KeyboardInterrupt: 

In [66]:
total = 0
correct = 0
for data in testloader:
    images, labels =data
    print(images.shape, labels.shape) # 8x3x32x32 / 8
    outputs = my_net(Variable(images))
    print(outputs.shape)# 8x10
    _, predicted = torch.max(outputs.data, 1) 
    print(predicted.shape)# 8x10 -> 8 
    total += labels.size(0)
    print(labels.size(0), total) #8을 계속 더해줌
    correct += (predicted == labels).sum() # 예측값이랑 label(둘다 size: 8)이 같으면 더해라
    
print('test images 10000의 accuracy: %f'%(100*correct/total))

torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 16
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 24
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 32
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 40
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 48
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 56
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 64
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 72
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 80
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 88
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8]

torch.Size([8, 10])
torch.Size([8])
8 1008
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1016
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1024
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1032
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1040
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1048
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1056
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1064
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1072
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1080
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1088
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1096
torch.Size(

torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1800
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1808
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1816
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1824
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1832
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1840
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1848
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1856
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1864
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1872
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 1880
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size(

torch.Size([8, 10])
torch.Size([8])
8 2584
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 2592
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 2600
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 2608
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 2616
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 2624
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 2632
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 2640
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 2648
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 2656
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 2664
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 2672
torch.Size(

torch.Size([8, 10])
torch.Size([8])
8 3392
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 3400
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 3408
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 3416
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 3424
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 3432
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 3440
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 3448
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 3456
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 3464
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 3472
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 3480
torch.Size(

...

torch.Size([8, 10])
torch.Size([8])
8 5752
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 5760
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 5768
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 5776
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 5784
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 5792
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 5800
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 5808
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 5816
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 5824
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 5832
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 5840
torch.Size(

torch.Size([8])
8 6760
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 6768
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 6776
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 6784
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 6792
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 6800
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 6808
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 6816
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 6824
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 6832
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 6840
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 6848
torch.Size([8, 3, 32, 32]) torc

torch.Size([8, 10])
torch.Size([8])
8 7560
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 7568
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 7576
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 7584
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 7592
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 7600
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 7608
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 7616
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 7624
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 7632
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 7640
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 7648
torch.Size(

torch.Size([8, 10])
torch.Size([8])
8 8440
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8448
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8456
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8464
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8472
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8480
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8488
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8496
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8504
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8512
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8520
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 8528
torch.Size(

torch.Size([8, 10])
torch.Size([8])
8 9304
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 9312
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 9320
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 9328
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 9336
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 9344
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 9352
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 9360
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 9368
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 9376
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 9384
torch.Size([8, 3, 32, 32]) torch.Size([8])
torch.Size([8, 10])
torch.Size([8])
8 9392
torch.Size(

In [73]:
a = torch.rand(8,10)
print(a)
print(a.shape)

tensor([[0.1231, 0.9310, 0.7411, 0.1804, 0.0477, 0.0213, 0.6539, 0.8399, 0.0548,
         0.0536],
        [0.7414, 0.0246, 0.1948, 0.7844, 0.1024, 0.3297, 0.7817, 0.3900, 0.1024,
         0.2759],
        [0.8375, 0.1431, 0.8255, 0.1091, 0.0136, 0.4678, 0.0736, 0.0487, 0.9467,
         0.5880],
        [0.9860, 0.3138, 0.5934, 0.3530, 0.8922, 0.6084, 0.7731, 0.6452, 0.9821,
         0.3582],
        [0.0551, 0.8978, 0.7637, 0.9652, 0.9807, 0.8146, 0.0404, 0.7098, 0.2540,
         0.9759],
        [0.3401, 0.4319, 0.7882, 0.1621, 0.6926, 0.6610, 0.6764, 0.1075, 0.5336,
         0.4983],
        [0.9092, 0.4998, 0.6218, 0.5864, 0.2665, 0.2758, 0.7724, 0.2498, 0.6513,
         0.1436],
        [0.9154, 0.2030, 0.4166, 0.2260, 0.6542, 0.9253, 0.6812, 0.8400, 0.8085,
         0.0749]])
torch.Size([8, 10])


In [74]:
c= torch.max(a,1) #-> 8X10이 8짜리 tensor 2개가 됨(독립된 tesnor)
print(c)
_, b= torch.max(a,1)
print(b)
print(b.shape)

(tensor([0.9310, 0.7844, 0.9467, 0.9860, 0.9807, 0.7882, 0.9092, 0.9253]), tensor([1, 3, 8, 0, 4, 2, 0, 5]))
tensor([1, 3, 8, 0, 4, 2, 0, 5])
torch.Size([8])


In [ ]:
class my_dataset(torch.utils.data.Dataset): # 나만의 data set 을 만드는 법
    def __init__(self,x,transforms=None): # 데이터 셋을 가져와서 선처리를 해준다
    def __len__(self):# 데이터 셋의 길이를 적어준다. (한개 반환)
    def __getitem__(self,idx):#데이터 셋에서 한 개의 데이터를 가져오는 함수를 정의한다

In [125]:
class model1(nn.Module):
    def __init__(self):
        super(model1,self).__init__()
        
        self.layer1 = nn.Sequential(
        nn.Conv2d(3,32,5),
        nn.ReLU(inplace = True),
        nn.Conv2d(32,64,3),
        nn.ReLU(inplace = True),
        nn.Conv2d(64,128,3),
        nn.Conv2d(128,128,3),
        nn.ReLU(inplace = True),
        nn.Conv2d(128,256,3),
        nn.MaxPool2d(2)
        )
        self.layer2_1 = nn.Sequential(
        nn.Conv2d(256,512,7,1,2),
        nn.Conv2d(512,64,1),
        nn.MaxPool2d(2)
        )
        self.layer2_2 = nn.Sequential(
        nn.Conv2d(256,512,5,1,1),
        nn.Conv2d(512,64,1),
        nn.MaxPool2d(2)
        )
        self.layer2_3 = nn.Sequential(
        nn.Conv2d(256,512,3),
        nn.Conv2d(512,64,1),
        nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
        nn.Linear(3*64*4*4,1024),
        nn.ReLU(True),
        nn.Linear(1024,10)
        )

    def forward(self,x):
        print(x.data.shape)
        x = self.layer1(x)
        x1 = self.layer2_1(x)
        x2 = self.layer2_2(x)
        x3 = self.layer2_3(x)
        x = torch.cat((x1,x2,x3),1) #1번 dimention을 합쳐주세요
        x = x.view(x.shape[0],-1) # batch size는 두고 나머지는 한줄로 펴주세요
        x = self.fc(x)
        return x
            
            


In [145]:
a = torch.rand(1,3,32,32)
print(a.shape)
a = Variable(a)
print(a.shape)

torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])


In [165]:
my_net1 = model1()
out = my_net1(a)
print(out.shape)
print(out)

torch.Size([1, 3, 32, 32])
torch.Size([1, 10])
tensor([[-0.0134, -0.0255,  0.0114, -0.0165, -0.0055,  0.0106, -0.0119,  0.0016,
          0.0450,  0.0256]], grad_fn=<AddmmBackward>)


In [166]:
torch.save(my_net1.state_dict(),'./cnn.pth')

In [167]:
my_net2 = model1()
my_net2.load_state_dict(torch.load('./cnn.pth'))
out = my_net2(a)
print(out)


torch.Size([1, 3, 32, 32])
tensor([[-0.0134, -0.0255,  0.0114, -0.0165, -0.0055,  0.0106, -0.0119,  0.0016,
          0.0450,  0.0256]], grad_fn=<AddmmBackward>)


In [168]:
my_net3 = model1()
out = my_net3(a)
print(out)

torch.Size([1, 3, 32, 32])
tensor([[ 0.0180,  0.0160,  0.0017, -0.0007,  0.0034,  0.0044,  0.0303,  0.0003,
          0.0210, -0.0069]], grad_fn=<AddmmBackward>)
